<a href="https://colab.research.google.com/github/adityaguin/AI_Honors_Course_Proj/blob/master/AUGMENTATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keybert flair lxml beautifulsoup4 selenium

     |████████████████████████████████| 401 kB 3.7 MB/s 
     |████████████████████████████████| 968 kB 26.0 MB/s 
     |████████████████████████████████| 79 kB 5.4 MB/s 
     |████████████████████████████████| 229 kB 40.5 MB/s 
     |████████████████████████████████| 51 kB 4.6 MB/s 
     |████████████████████████████████| 4.0 MB 33.4 MB/s 
     |████████████████████████████████| 1.2 MB 23.6 MB/s 
     |████████████████████████████████| 77 kB 4.8 MB/s 
     |████████████████████████████████| 596 kB 41.3 MB/s 
     |████████████████████████████████| 6.5 MB 41.3 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 1.6 MB 48.9 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2 MB 50.6 MB/s 
     |████████████████████████████████| 46 kB 3.1 MB/s 
     |█████████

In [ ]:
import torch
import pandas as pd
import bz2
import json
from flair.models import TextClassifier
from flair.data import Sentence
from selenium import webdriver as dd
from bs4 import BeautifulSoup
import pandas as pd
import requests
from os import getcwd
import time
from keybert import KeyBERT
from torch.utils.data import Dataset, DataLoader
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
import spacy
from sklearn.model_selection import train_test_split

## RETURN SUBJECT OF SENTENCE

In [ ]:
nlp = spacy.load("en_core_web_sm")

def returnSubject(sentence):
  list = []
  sen = nlp(sentence)
  for token in sen:
    if token.pos_ == "PROPN" or token.dep_ == 'nsubj':
      list.append(token.text)
    # print(token.text, token.pos_, token.dep_)
  return list

def returnProperN(sentence):
  list = []
  sen = nlp(sentence)
  for token in sen:
    if token.pos_ == "PROPN":
      list.append(token.text)
  return list

#KEYWORD EXTRACTION

In [ ]:
# 03/02/2022 testing out keyword extraction using keyBERT

message = "I bet Clinton can't wait to attend Lewinsky's housewarming party. #scandalous"
print(returnSubject(message))

k_model = KeyBERT()
keyboard = k_model.extract_keywords(message)
keyBertList = k_model.extract_keywords(message, keyphrase_ngram_range=(1, 1), stop_words=None)
for x in keyBertList:
  print(x)


newKeyMessage = [term for term in returnSubject(message)]


def changeMessage(message, finalRelevant):
  # find proper noun in tweet, append the finalRelevant after it
  precedingSentence = returnProperN(message)
  print(precedingSentence)
  precedingSentence += finalRelevant
  return " ".join(precedingSentence) + ". " + message  
  
# newKeyMessage.appe

['I', 'Clinton', 'Lewinsky']
('lewinsky', 0.6614)
('clinton', 0.5002)
('scandalous', 0.4429)
('party', 0.2949)
('housewarming', 0.2947)


# WEBSCRAPING

In [ ]:

dset = set()

def google():
    query = 'Lewinsky+Clinton+Scandalous'
    url = 'https://www.google.com/search?q=' + query
    html_text = (requests.get(url)).text
    soup = BeautifulSoup(html_text, 'lxml')
    descrip = soup.find_all('div', class_='BNeawe s3v9rd AP7Wnd')
    for d in descrip:
        dset.add(d.text)

# Consider phrases of 2 words in the set for future
google()
sum = 0;
d_str = ''.join(dset)
word_set = set(d_str.split())
relavent_words = []
sia = TextClassifier.load('sentiment') 
labels = {'POSITIVE': [], 'NEGATIVE': []} 
sentence = Sentence(d_str)
for ord in word_set:
  Ord = Sentence(ord)
  sia.predict(Ord)
  lbl = Ord.get_label()
  labels[lbl.value].append((ord, lbl.score))
labels['POSITIVE'].sort(key=lambda x: x[1], reverse=True)
labels['NEGATIVE'].sort(key=lambda x: x[1], reverse=True)
# print(labels['POSITIVE'])
# print(labels['NEGATIVE'])
sia.predict(sentence)
  
relavent_words = labels[sentence.get_label().value]
   
# print(relavent_words) 

finalRelevant = []
for i in range(0,6):
  finalRelevant.append(relavent_words[i][0])

print(finalRelevant)
print(changeMessage(message, finalRelevant))



2022-04-08 18:06:12,536 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt
['burned', 'broke', 'denial', 'inappropriate”', 'lie', 'scandal']
['Clinton', 'Lewinsky']
Clinton Lewinsky burned broke denial inappropriate” lie scandal. I bet Clinton can't wait to attend Lewinsky's housewarming party. #scandalous


In [ ]:
print(relavent_words[1])

[('lost', 0.9932103753089905), ('case', 0.9902495741844177), ('alleged', 0.9867604970932007)]


In [ ]:
d_str = ''.join(dset)
d_str

In [ ]:
lbl = Ord.get_label()
lbl.shortstring

'"Sep"/POSITIVE'